# Teams_Predicao

Este notebook é dedicado à predição de dados das tabelas **Teams1** e **Teams2** fornecidas pela IBM, que contém dados detalhados sobre os times, incluindo informações como quantidade de gols do time, cartões amarelos que têm no total, entre outros. O objetivo deste notebook é unir ambas as tabelas e assim confirmar as hipóteses e prever mais informações com os dados dessas tabelas.

## Objetivo

O objetivo deste notebook é fornecer uma predição de dados dos times da série A do Campeonato Brasileiro, ajudando a identificar padrões e tendências que possam ser úteis para diversas aplicações, como previsões de resultados e desempenho dos times.

## Como Usar Este Notebook

1. **Configuração do Ambiente**:
   - **Google Colab**: No Google Colab, será necessário fazer o upload das tabelas para o Google Drive e montar o drive no notebook.
   - **Localmente**: Se for rodar o notebook localmente, é necessário baixar as tabelas e colocá-las no mesmo diretório do notebook ou ajustar os caminhos dos arquivos conforme necessário.

2. **Instalação de Dependências**:
   - Certifique-se de que todas as bibliotecas necessárias estão instaladas. Você pode instalar as dependências utilizando o seguinte comando:
     ```python
     !pip install -r requirements.txt
     ```

3. **Execução do Notebook**:
   - Antes de rodar esse notebook, execute por completo os notebooks `teams2_processado.ipynb` e `teams1_processado.ipynb` para que sejam exportadas em suas células as respectivas tabelas contendo os dados tratados, que serão utilizados neste notebook.
   - Siga as células de código sequencialmente para garantir que todas as etapas sejam executadas corretamente. Cada seção do notebook está organizada para facilitar a compreensão e a análise dos dados.

## Nesse Notebook Será Abordado

1. **União das tabelas**:
   - Exclusão das colunas em comum das tabelas `Teams1` e `Teams2`
   - Merge dessas tabelas em torno de uma coluna base

2. **Modelagem para o problema**:
   - Modelagem para o problema - proposta das features;
   - Organização dos dados - como os dados foram organizados;
   - Apresentar o modelo candidato - qual modelo estamos utilizando e para que;
   - Métricas relacionadas ao modelo - avaliação de desempenho.

obs1: A hipótese abordada nesse modelo pode ser encontrada no [Notebook Hipótese 3 - Seção 3](../../notebooks/hipoteses/teams_hipoteses.ipynb)

obs2: Antes de rodar esse notebook, certifique-se de que você rodou esses outros dois:

   * [Notebook Tratamento Teams1](../../notebooks/pre_processamento/teams1_processado.ipynb)

   * [Notebook Tratamento Teams2](../../notebooks/pre_processamento/teams2_processado.ipynb)

# Dependências
Para rodar o notebook de forma local, é recomendado que inicie uma venv (ambiente virtual) e instale as dependências.

Se estiver utilizando o Google Colab, pule esta etapa.


In [ ]:
# Instala as dependências
!pip install -r requirements.txt

# Importando bibliotecas

Aqui é importado as dependências necessárias para a executação do projeto.

In [ ]:
import pandas as pd #para ler, visualizar e printar infos do df
import matplotlib.pyplot as plt #para construir e customizar gráficos
import seaborn as sns #para visualizar uns gráficos
import numpy as np #numpy porque é sempre bom importar numpy
import math #para executar operações matemáticas
import shap

from scipy import stats #para executar testes estatísticos
from scipy.stats.mstats import winsorize #para realizar a winsorização

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder #para realizar o pré-processamento de dados
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_absolute_error, r2_score, mean_squared_error, median_absolute_error, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# 1 União das tabelas

Nesta etapa, realizamos a integração de dois conjuntos de dados (Teams1 e Teams2) para obter uma visão mais abrangente dos times.

## 1.1 Carregando os datasets

Feita a importação dos arquivos para leitura dos dados. Esses arquivos foram exportados dos noteboos `teams1_tratado.ipynb` e `teams2_tratado.ipynb`

Na leitura de dados vamos conseguir carregar os dados para depois serem análisados

* O arquivo CSV contendo as estatísticas das equipes é lido e armazenado na variável teams
Depois que executar esse código vamos conseguir ler o arquivo e 'armazenar' na variável.

`teams1_tratado.ipynb` e `teams2_tratado.ipynb`

Na leitura de dados vamos conseguir carregar os dados para depois serem análisados

* O arquivo CSV contendo as estatísticas das equipes é lido e armazenado na variável teams
Depois que executar esse código vamos conseguir ler o arquivo e 'armazenar' na variável.

In [ ]:
teams1 = pd.read_csv('../../notebooks/data/tratado/teams1_tratado.csv') 
teams2 = pd.read_csv('../../notebooks/data/tratado/teams2_tratado.csv')

## 1.2 Verificando as colunas do Teams1

Aqui faremos a verificação das colunas que o arquivo `teams1_tratado.ipynb` contém para evitar o merge de colunas repetidas, visto que ambas as tabelas possuem dados que se complementam.

In [ ]:
#Coloca em uma variável chamada colunas categóricas todas as colunas que tem dados do tipo categórico, como nome dos times
colunas_categoricas = teams1.select_dtypes(include='object').columns
#Coloca em uma variável chamada colunas numéricas todas as colunas que tem dados do tipo numérico, como o o número de gols que fez em casa
colunas_numericas = teams1.drop(colunas_categoricas, axis=1).columns

# Imprime a lista de colunas categóricas.
print(f'Há {len(colunas_categoricas)} Colunas Categóricas: {list(colunas_categoricas)}')

# Imprime a lista de colunas numéricas.
print(f'Há {len(colunas_numericas)} Colunas Numéricas: {list(colunas_numericas)}')

## 1.3 Verificando as colunas do Teams2

Aqui faremos a verificação das colunas que o arquivo `teams2_tratado.ipynb` contém para evitar o merge de colunas repetidas, visto que ambas as tabelas possuem dados que se complementam.

In [ ]:
#Coloca em uma variável chamada colunas categóricas todas as colunas que tem dados do tipo categórico, como nome dos times
colunas_categoricas = teams2.select_dtypes(include='object').columns
#Coloca em uma variável chamada colunas numéricas todas as colunas que tem dados do tipo numérico, como o o número de gols que fez em casa
colunas_numericas = teams2.drop(colunas_categoricas, axis=1).columns

# Imprime a lista de colunas categóricas.
print(f'Há {len(colunas_categoricas)} Colunas Categóricas: {list(colunas_categoricas)}')

# Imprime a lista de colunas numéricas.
print(f'Há {len(colunas_numericas)} Colunas Numéricas: {list(colunas_numericas)}')

## 1.4 Verificação de colunas em comum

Após imprimir as colunas de ambos os arquivos (`teams1_tratado.ipynb` e `teams2_tratado.ipynb`) podemos verificar que existem algumas colunas em comum neles, sendo elas: 'minutes_per_goal_scored_home', 'minutes_per_goal_scored_away', 'minutes_per_goal_conceded_home', 'minutes_per_goal_conceded_away' e 'common_name_encoded'

A última coluna, 'common_name_encoded', será utilizada como referência para a união das tabelas, por isso ela não sera excluida de uma das tabelas (para evitar a duplicação), mas, como as outras possuem a mesma informação em ambas as tabelas, elas serão excluidas em uma delas

In [ ]:
# Comparar as colunas dos dois datasets para ver quais estão em comum
colunas_comuns = set(teams1.columns).intersection(set(teams2.columns))

# Exibir as colunas em comum
print("As colunas em comum são:", colunas_comuns)

## 1.5 Exclusão de colunas

Como abordado na seção 1.3, é importante identificar essas colunas duplicadas nos datasets para que elas não atrapalhem no futuro do modelo, sendo assim, por elas serem iguais, optamos por excluir elas de um dataset, assim não ficando duplicadas na hora do merge.

In [56]:
apagar = ['minutes_per_goal_scored_home', 
    'minutes_per_goal_scored_away', 
    'minutes_per_goal_conceded_home', 
    'minutes_per_goal_conceded_away'
]

# Apagando as colunas do DataFrame
teams1 = teams1.drop(columns=apagar, errors='ignore')

## 1.6 Merge das tabelas

Nesta etapa, realizamos a fusão (merge) entre os dois datasets, teams1 e teams2, para combinar as informações compartilhadas entre eles. Isso é útil quando você tem dois datasets que contêm informações complementares sobre os mesmos registros (neste caso, sobre times).

*Função utilizada: `pd.merge()`
*Parâmetro on: Especifica a coluna comum entre os dois datasets que será usada para fazer a junção. Aqui, a coluna `common_name_encoded` é o identificador comum entre os dois datasets, o que significa que estamos combinando os dados baseados nos nomes dos times codificados.

In [ ]:
dados = pd.merge(teams1, teams2, on=['common_name_encoded', 'common_name'])
dados.head()

# 1.7 Verificação de Colunas Duplicadas

Após realizar a operação de merge entre as tabelas teams1 e teams2, é importante verificar se o dataset resultante contém colunas duplicadas. Colunas duplicadas podem surgir quando ambos os datasets possuem colunas com o mesmo nome. Por padrão, a função merge adiciona sufixos (_x e _y) para diferenciar as colunas duplicadas, por isso não podemos fazer um simples if que passa verificando os nomes das colunas

In [ ]:
# Extrair as colunas antes do merge para verificar duplicadas
colunas_teams1 = set(teams1.columns)
colunas_teams2 = set(teams2.columns)

# Identificar colunas que estão presentes em ambos os datasets
colunas_comuns = colunas_teams1.intersection(colunas_teams2)

print(f"Colunas que estavam duplicadas antes do merge: {colunas_comuns}")

# Verificar as colunas após o merge para garantir que os sufixos estão presentes
colunas_com_sufixo = [col for col in dados.columns if col.endswith('_x') or col.endswith('_y')]

print(f"Colunas com sufixo adicionadas após o merge: {colunas_com_sufixo}")

## 1.8 Exportação dos Dados

Após realizar a junção e o tratamento dos dados provenientes das tabelas teams1 e teams2 fazemos a exportação para salvar essa coluna nos arquivos locais do PC e não termos que rodar os notebooks de tratamento do teams1 e teams2 sempre que quisermos executar esse código

In [59]:
dados.to_csv('../../notebooks/data/tratado/teams_complete.csv', index=False) 

## 1.9 Times codificados

Aqui colocamos os nomes dos times e sua respectiva codificação para consulta ao longo do código caso preciso.

| common_name       | common_name_encoded |
|-------------------|---------------------|
| Atlético GO       | 0                   |
| Atlético Mineiro  | 1                   |
| Atlético PR       | 2                   |
| Bahia             | 3                   |
| Botafogo          | 4                   |
| Bragantino        | 5                   |
| Corinthians       | 6                   |
| Criciúma          | 7                   |
| Cruzeiro          | 8                   |
| Cuiabá            | 9                   |
| Flamengo          | 10                  |
| Fluminense        | 11                  |
| Fortaleza         | 12                  |
| Grêmio            | 13                  |
| Internacional     | 14                  |
| Juventude         | 15                  |
| Palmeiras         | 16                  |
| São Paulo         | 17                  |
| Vasco da Gama     | 18                  |
| Vitória           | 19                  |


# 2 Desenvolvimento do Modelo Preditivo

Nesta seção, abordamos o processo de desenvolvimento dos modelo preditivos, desde a seleção das features mais relevantes para realizar as previsões. Descrevemos como os dados foram organizados e preparados para alimentar o modelo e detalhamos a abordagem utilizada para a modelagem. Além disso, apresentamos o modelo candidato selecionado, explicando sua função e como ele foi ajustado para resolver o problema proposto. Por fim, discutimos as métricas utilizadas para avaliar o desempenho do modelo, oferecendo uma análise clara de sua eficácia preditiva.

## 2.1 Hipótese: vitórias até o final do campeonato

#### 2.1.1 Descrição:
A hipótese geral desse modelo de Random Forest que criamos é baseada na ideia de que o desempenho passado de um time, medido por diversas métricas, pode prever o número de vitórias futuras até o final do campeonato. O modelo faz isso analisando variáveis-chave, como a posição na liga, saldo de gols, pontos por jogo, e outras estatísticas de desempenho, para identificar padrões que possam se repetir nas rodadas restantes.

#### 2.1.2 Hipótese do Modelo:
Premissa Principal: O desempenho anterior de um time é um bom indicador do desempenho futuro. Times que já acumularam vitórias, mantêm boas médias de pontos por jogo e apresentam uma boa defesa (representada por poucas derrotas e gols sofridos) têm maiores chances de continuar vencendo nas rodadas restantes.

##### 2.1.3 Justificativa da Premissa:
Posição na liga e pontos por jogo refletem a consistência do time ao longo do campeonato. Esses fatores indicam o equilíbrio entre vitórias, empates e derrotas e sugerem uma tendência de como o time pode se comportar no futuro.
Saldo de gols e gols sofridos medem a eficiência do time, tanto no ataque quanto na defesa. Times com saldo de gols positivo ou uma defesa forte têm mais chances de vencer os jogos restantes, o que torna essas métricas bons preditores.
Histórico de vitórias até o momento oferece uma visão direta do quanto o time tem sido competitivo. Quanto mais vitórias, maior a probabilidade de manter esse desempenho.
Abordagem Técnica: O modelo de Random Forest foi escolhido porque é robusto contra outliers e flexível o suficiente para capturar relações complexas entre as diversas variáveis preditoras. A Random Forest cria múltiplas árvores de decisão, onde cada árvore faz previsões baseadas em subconjuntos diferentes das variáveis. O modelo final é a agregação dessas previsões, o que ajuda a minimizar erros e melhorar a precisão das estimativas.

Previsão Ajustada: O modelo também foi ajustado para considerar o número de rodadas restantes no campeonato, de modo que as previsões levem em conta quantos jogos ainda faltam. Dessa forma, ele prevê quantas vitórias o time conseguirá nas rodadas que restam com base em seu desempenho até agora.

## 2.2 Modelagem para o Problema (Seleção das Principais Features)

Nesta fase, utilizamos o **Random Forest Regressor** para identificar as variáveis que mais influenciam o número de vitórias futuras de uma equipe em um campeonato de futebol. O **Random Forest** é ideal para esse tipo de problema, pois lida bem com dados complexos e não lineares, além de ser robusto a outliers. O modelo nos permite captar padrões importantes no desempenho passado das equipes, considerando tanto aspectos ofensivos quanto defensivos.

### 2.2.1 Proposta de Features e Linha de Raciocínio

#### Premissa da Modelagem:

Partimos da premissa de que o desempenho passado de uma equipe ao longo de um campeonato é um bom indicador de seu desempenho futuro. Portanto, a seleção das features foi orientada a capturar elementos cruciais que afetam diretamente as vitórias de uma equipe, como posição na liga, saldo de gols, posse de bola, e outros indicadores de consistência de desempenho.

### 2.2.2 Proposta das Features:

##### Variáveis Utilizadas:
Abaixo estão as principais features selecionadas para o modelo de previsão de vitórias:
- **Desempenho em Rodadas Anteriores**: `matches_played`, `matches_played_home`, `matches_played_away`, `draws`, `losses`, `points_per_game`, `points_per_game_home`, `points_per_game_away`.
- **Posição e Estatísticas de Desempenho Geral**: `league_position`, `goals_scored`, `goals_conceded`, `goal_difference`, `total_goal_count`, `shots_on_target`, `shots_off_target`.
- **Posse de Bola e Controle do Jogo**: `average_possession`.

### 2.2.3 Explicação do Raciocínio:

1. **Desempenho Passado**: Métricas como `matches_played`, `points_per_game`, e `league_position` fornecem uma visão clara da consistência de um time ao longo da temporada. Equipes que mantêm um bom desempenho regular geralmente continuam acumulando vitórias em rodadas futuras.

2. **Eficiência Ofensiva e Defensiva**: `goals_scored`, `goals_conceded`, e `goal_difference` são indicadores diretos de como a equipe se comporta tanto no ataque quanto na defesa. Times com saldo de gols positivo e uma boa eficiência defensiva têm maior probabilidade de manter esse padrão nas partidas seguintes, o que os torna fortes candidatos a vitórias.

3. **Controle de Jogo**: Variáveis como `average_possession` e o número de chutes (`shots_on_target`, `shots_off_target`) refletem a capacidade de uma equipe controlar o jogo e criar oportunidades de gol. Times que controlam mais a posse e atacam com frequência tendem a prevalecer em seus jogos.

### 2.2.4 Normalização das Variáveis:

Além da seleção das features, aplicamos o método **StandardScaler** para normalizar as variáveis numéricas. Isso é fundamental, pois evita que variáveis com diferentes magnitudes (por exemplo, número de chutes versus porcentagem de posse de bola) influenciem de maneira desproporcional o desempenho do modelo. A normalização garante que todas as features estejam na mesma escala, proporcionando uma análise justa e equilibrada.

### 2.2.5 Justificativa da Premissa:

A combinação das features selecionadas se justifica pelos seguintes fatores:
- **Posição na Liga e Pontos por Jogo**: Esses indicadores são fundamentais para captar a regularidade de um time ao longo do campeonato, mostrando um equilíbrio entre vitórias, empates e derrotas.
- **Saldo de Gols e Gols Sofridos**: Estas variáveis refletem a eficiência da equipe tanto no ataque quanto na defesa. Times que marcam mais gols do que sofrem tendem a ter mais sucesso nas rodadas futuras.
- **Histórico de Vitórias**: Quanto mais vitórias uma equipe acumula até o momento, maior a probabilidade de que continue vencendo, já que isso reflete uma trajetória positiva e consistente.

### 2.2.6 Abordagem Técnica:

O **Random Forest Regressor** foi escolhido como modelo base por várias razões:
- **Robustez**: O Random Forest é robusto contra outliers e dados ruidosos. Isso é importante, especialmente em campeonatos longos, onde algumas equipes podem ter performances atípicas em certos jogos.
- **Flexibilidade**: O modelo pode capturar relações complexas e não lineares entre as variáveis preditoras, o que é importante para um esporte como o futebol, onde muitos fatores influenciam os resultados.
- **Redução de Overfitting**: Como o Random Forest cria múltiplas árvores e faz uma agregação das previsões, ele reduz o risco de overfitting, proporcionando previsões mais generalizáveis. Isso é essencial para manter a precisão do modelo ao lidar com novos dados.

Essa abordagem, combinada com a seleção de features que capturam tanto aspectos ofensivos quanto defensivos das equipes, oferece uma visão abrangente do desempenho de uma equipe, permitindo que o modelo faça previsões precisas e confiáveis.

### 2.2.7 Previsão Ajustada:

O modelo foi ajustado para considerar o número de rodadas restantes no campeonato. Ao prever o número de vitórias de uma equipe, o modelo leva em conta quantos jogos ainda faltam, ajustando a previsão com base no desempenho atual da equipe e no número de rodadas restantes. Isso proporciona uma visão mais realista e prática das chances de uma equipe continuar vencendo até o final da competição.

Essa combinação de técnicas e justificativas coloca o **Random Forest Regressor** como uma ferramenta eficaz para prever vitórias futuras em campeonatos de futebol, utilizando uma seleção de features que capturam os aspectos mais críticos do desempenho de uma equipe.

In [ ]:
# Definir as colunas significativas para a predição
significant_columns = ['matches_played', 'matches_played_home', 'matches_played_away', 
                       'draws', 'draws_home', 'draws_away', 'losses', 'losses_home', 'losses_away', 'points_per_game', 
                       'points_per_game_home', 'points_per_game_away', 'league_position', 'goals_scored', 'goals_conceded', 
                       'goal_difference', 'total_goal_count', 'shots_on_target', 'shots_off_target', 'average_possession']

# Preparar a matriz de features (X) e a variável alvo (y)
X = dados[significant_columns]
y = dados['wins']

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Treinar o modelo Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o modelo com métricas
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
medae = median_absolute_error(y_test, y_pred)

# Prever as vitórias para um time específico
def prever_vitorias(team_input):
    # Verificar se a entrada é um número (ID) ou string (nome)
    if isinstance(team_input, int):
        team_data = dados[dados['common_name_encoded'] == team_input]
        team_name = dados[dados['common_name_encoded'] == team_input]['common_name'].values[0]
    elif isinstance(team_input, str):
        team_data = dados[dados['common_name'] == team_input]
        team_name = team_input
    else:
        print("Entrada inválida! Insira um ID numérico ou o nome do time.")
        return

    # Verificar se o time tem dados suficientes
    if not team_data.empty:
        # Número total de rodadas no campeonato
        total_rounds = 38
        matches_played = team_data['matches_played'].values[0]

        # Calcular rodadas restantes
        remaining_rounds = total_rounds - matches_played

        # Previsão de vitórias até o momento
        team_X = team_data[significant_columns]
        predicted_wins = model.predict(team_X)

        # Ajustar a previsão com base nas rodadas restantes
        predicted_wins_adjusted = (predicted_wins / matches_played) * remaining_rounds

        print(f"Previsão de vitórias para o time {team_name} até o final do campeonato: {predicted_wins_adjusted[0]:.2f}")
    else:
        print(f"Sem dados para o time {team_name}")

# Exemplo de uso: prever vitórias para um time específico
prever_vitorias(16)  # Usar ID do time
prever_vitorias('Atlético GO')  # Usar nome do time

# Exibir as métricas para comparação
(mae, r2, mse, rmse, medae)


### 2.3 Ajuste de Hiperparâmetros com Grid Search

Nesta etapa, utilizamos o **Grid Search** para encontrar a melhor combinação de hiperparâmetros para o modelo **Random Forest Regressor**. O ajuste de hiperparâmetros é essencial para maximizar a performance do modelo, já que diferentes configurações podem influenciar diretamente o resultado final das previsões. Aqui, o **GridSearchCV** testa exaustivamente várias combinações de hiperparâmetros e seleciona a que proporciona o melhor desempenho com base em uma métrica de avaliação específica.

#### 2.3.1 Hiperparâmetros Definidos:

- **n_estimators**: Controla o número de árvores na floresta. Mais árvores tendem a reduzir o erro de variância, mas aumentam o custo computacional. Testamos 50, 100, 150 e 200 árvores.
  
- **max_depth**: Define a profundidade máxima de cada árvore. Profundidades maiores permitem que as árvores capturem mais detalhes dos dados, mas podem levar ao overfitting. Testamos profundidades de 5, 10, 15 e 20.

- **min_samples_split**: Controla o número mínimo de amostras exigidas para dividir um nó. Valores menores tornam o modelo mais sensível a variações nos dados, enquanto valores maiores tornam as árvores mais generalizáveis. Os valores testados foram 2, 5 e 10.

- **min_samples_leaf**: Define o número mínimo de amostras em uma folha. Ajustar esse parâmetro ajuda a evitar a criação de folhas muito pequenas, o que pode levar a modelos muito específicos e pouco generalizáveis. Testamos os valores 1, 2 e 4.

- **bootstrap**: Indica se deve ser utilizada a amostragem com ou sem substituição ao construir as árvores. Quando `True`, o modelo usa amostragem com substituição, o que pode aumentar a robustez do modelo.

#### 2.3.2 Processo de Otimização:

Usamos o **GridSearchCV** para testar todas as combinações dos hiperparâmetros especificados. A métrica de avaliação escolhida foi o **erro quadrático médio negativo** (`neg_mean_squared_error`), que quantifica a diferença entre os valores reais e previstos. A validação cruzada (com `cv=5`) foi utilizada para garantir que o modelo não superajustasse os dados de treino. Além disso, utilizamos todos os núcleos disponíveis da CPU (`n_jobs=-1`) para acelerar o processo de treinamento.

#### 2.3.3 Resultados:

Após a execução do **Grid Search**, obtemos a melhor combinação de hiperparâmetros (`best_params_rf`) e a melhor pontuação (`best_score_rf`) com base na métrica definida. Isso nos permite otimizar o modelo e obter previsões mais precisas para o número de vitórias futuras de uma equipe.

In [ ]:
# Definir os hiperparâmetros a serem ajustados no Grid Search
param_grid_rf = {
    'n_estimators': [50, 100, 150, 200],  # Diferentes números de árvores na floresta
    'max_depth': [5, 10, 15, 20],         # Diferentes profundidades máximas das árvores
    'min_samples_split': [2, 5, 10],      # Número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4],        # Número mínimo de amostras em uma folha
    'bootstrap': [True, False]            # Usar ou não amostragem com substituição
}

# Inicializar o modelo Random Forest
rf_model = RandomForestRegressor(random_state=42)

# Inicializar o GridSearchCV para Random Forest Regressor
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, 
                              scoring='neg_mean_squared_error', n_jobs=-1, cv=5, verbose=1)

# Treinar o GridSearchCV
grid_search_rf.fit(X_train, y_train)

# Melhor combinação de hiperparâmetros
best_params_rf = grid_search_rf.best_params_

# Melhor pontuação
best_score_rf = grid_search_rf.best_score_

best_params_rf, best_score_rf


### 2.4 Aplicação dos Hiperparâmetros Otimizados e Avaliação do Modelo

Após a otimização dos hiperparâmetros com o **Grid Search**, aplicamos os melhores parâmetros encontrados ao modelo **Random Forest Regressor**. Essa etapa é fundamental para garantir que o modelo final seja o mais eficiente possível, utilizando a combinação de parâmetros que melhor se ajustou aos dados de treinamento durante a validação cruzada.

#### 2.4.1 Parâmetros Otimizados:

Os melhores hiperparâmetros identificados pelo **GridSearchCV** foram os seguintes:
- **bootstrap**: `False` — não será utilizada a amostragem com substituição ao construir as árvores.
- **max_depth**: `5` — a profundidade máxima das árvores será limitada a 5 níveis, evitando que o modelo se torne muito específico (overfitting).
- **min_samples_leaf**: `1` — cada folha final conterá, no mínimo, uma amostra.
- **min_samples_split**: `2` — um nó precisará ter pelo menos 2 amostras para ser dividido.
- **n_estimators**: `200` — o modelo terá 200 árvores, o que ajuda a capturar variações nos dados de forma mais robusta.
- **random_state**: `42` — utilizado para garantir a reprodutibilidade dos resultados.

#### 2.4.2 Treinamento do Modelo:

Com esses parâmetros otimizados, o modelo foi treinado utilizando os dados de treinamento. O conjunto de treino (`X_train` e `y_train`) foi utilizado para ajustar o modelo, que aprendeu a prever o número de vitórias com base nas features selecionadas.

#### 2.4.3 Previsão e Avaliação:

O modelo foi então aplicado ao conjunto de teste (`X_test`) para fazer previsões sobre as vitórias (`y_pred`). Para avaliar o desempenho, utilizamos diversas métricas:
- **MAE (Mean Absolute Error)**: Mede a média absoluta dos erros entre as previsões e os valores reais.
- **R² (Coeficiente de Determinação)**: Avalia o quão bem as previsões do modelo se ajustam aos valores reais.
- **MSE (Mean Squared Error)**: Calcula a média dos quadrados dos erros entre previsões e valores reais.
- **RMSE (Root Mean Squared Error)**: Raiz quadrada do MSE, oferece uma métrica mais interpretável no mesmo espaço das variáveis.
- **MedAE (Median Absolute Error)**: Mede a mediana dos erros absolutos, sendo menos sensível a outliers.

#### 2.4.4 Previsão Ajustada para um Time Específico:

Para prever o número de vitórias restantes de um time específico até o final do campeonato, utilizamos os dados do time (`team_data`) e ajustamos a previsão com base no número de rodadas restantes. O processo consiste em prever o número de vitórias até o momento e, em seguida, ajustar essa previsão com base no número de jogos restantes no campeonato.

Se os dados para o time estão disponíveis, o modelo prevê o número de vitórias para as rodadas futuras, ajustado proporcionalmente à quantidade de jogos restantes. Caso contrário, é exibida uma mensagem indicando a falta de dados.

#### 2.4.5 Resultados:

Por fim, são exibidas as métricas de avaliação para verificar a qualidade do modelo, e também a previsão ajustada de vitórias para o time selecionado, considerando as rodadas restantes do campeonato.

In [ ]:
# Definir os parâmetros encontrados pelo GridSearchCV
best_params = {
    'bootstrap': False,
    'max_depth': 5,
    'min_samples_leaf': 1,
    'min_samples_split': 2,
    'n_estimators': 200,
    'random_state': 42
}

# Aplicar os melhores parâmetros ao modelo Random Forest Regressor
model = RandomForestRegressor(**best_params)

# Treinar o modelo com os dados de treinamento
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o modelo com métricas
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
medae = median_absolute_error(y_test, y_pred)

# Prever as vitórias para um time específico
team_id = 16  
team_data = dados[dados['common_name_encoded'] == team_id]

# Verificar se o time tem dados suficientes
if not team_data.empty:
    # Número total de rodadas no campeonato
    total_rounds = 38
    matches_played = team_data['matches_played'].values[0]
    
    # Calcular rodadas restantes
    remaining_rounds = total_rounds - matches_played
    
    # Previsão de vitórias até o momento
    team_X = team_data[significant_columns]
    predicted_wins = model.predict(team_X)
    
    # Ajustar a previsão com base nas rodadas restantes
    predicted_wins_adjusted = (predicted_wins / matches_played) * remaining_rounds
    
    print(f"Previsão de vitórias para o time {team_id} até o final do campeonato:", predicted_wins_adjusted[0])
else:
    print(f"Sem dados para o time {team_id}")

# Exibir as métricas
(mae, r2, mse, rmse, medae)


### 2.5 Avaliação do Modelo com Validação Cruzada

Após ajustar o modelo com os melhores hiperparâmetros, aplicamos a **validação cruzada** para avaliar seu desempenho de maneira mais robusta e verificar sua generalização em diferentes subconjuntos dos dados. A validação cruzada com **5 folds** (divisões) é uma técnica amplamente utilizada, onde os dados são divididos em cinco partes, e o modelo é treinado em quatro dessas partes enquanto a quinta parte é usada para teste. Esse processo é repetido até que cada parte tenha sido utilizada como conjunto de teste uma vez.

#### 2.5.1 Cálculo da Métrica de Erro (MAE)

Utilizamos o **Mean Absolute Error (MAE)** como a métrica de avaliação, pois ela mede a média dos erros absolutos entre as previsões e os valores reais, fornecendo uma interpretação direta da magnitude dos erros.

O processo foi realizado da seguinte forma:

1. **Aplicação do Cross-Validation**: O modelo foi avaliado em 5 folds distintos dos dados, retornando um valor de MAE para cada um dos 5 testes.
   
2. **Conversão dos Valores**: Como o **cross_val_score** retorna o MAE negativo (convenção de otimização), invertemos os sinais para obter os valores positivos do erro absoluto.

3. **Cálculo da Média e Desvio Padrão**: Para uma visão geral do desempenho do modelo, calculamos:
   - **MAE Médio**: A média dos valores de erro absoluto, que fornece uma estimativa geral do desempenho do modelo.
   - **Desvio Padrão do MAE**: Mede a variabilidade entre os diferentes folds, indicando a consistência do modelo em diferentes subconjuntos dos dados.

#### 2.5.2 Resultados:

- **MAE Médio**: 0.268 — isso significa que, em média, as previsões do modelo desviam 0.268 unidades em relação aos valores reais. 

- **Desvio Padrão do MAE**: 0.098 — o baixo desvio padrão indica que o modelo tem uma performance consistente entre os diferentes subconjuntos de dados, sugerindo uma boa capacidade de generalização.

Esses resultados mostram que o modelo **Random Forest Regressor** ajustado com os melhores hiperparâmetros apresenta um desempenho sólido e estável na previsão do número de vitórias, com erros relativamente baixos e consistentes.

In [ ]:
# Aplicar validação cruzada (5 folds) ao modelo com os melhores parâmetros
mae_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')

# Converter os resultados de MAE para valores positivos
mae_scores = -mae_scores

# Calcular a média do MAE e seu desvio padrão
mean_mae = np.mean(mae_scores)
std_mae = np.std(mae_scores)

# Exibir os resultados
print(f'MAE médio: {mean_mae}')
print(f'Desvio padrão do MAE: {std_mae}')

### 2.6 Importância das Features no Modelo:

#### 2.6.1 Descrição: 
Nesta seção, avaliamos a importância de cada variável no desempenho do modelo preditivo. O **Random Forest** oferece a vantagem de destacar quais features têm mais impacto nas previsões, ajudando a identificar os fatores mais influentes no resultado. Essa análise é crucial para priorizar variáveis relevantes em futuras otimizações e simplificar o modelo, descartando aquelas com pouca ou nenhuma contribuição significativa.

#### 2.6.2 Principais Features no Modelo:

* **Posição na Liga (25,5%)**: A variável mais importante no modelo é a posição na liga, o que é intuitivo, já que reflete a combinação de vitórias, empates, derrotas e saldo de gols. Times em posições mais altas da tabela têm um desempenho geral superior, o que aumenta suas chances de continuar vencendo nas rodadas futuras. A posição na liga resume o histórico de desempenho da equipe, sendo um dos fatores mais preditivos.

* **Pontos por Jogo (24,1%)**: O segundo fator mais importante é a média de pontos por jogo. Essa métrica demonstra a capacidade de um time em converter seu desempenho em resultados concretos ao longo da temporada. Times com uma alta média de pontos por jogo são mais consistentes e têm maior probabilidade de manter esse nível nas rodadas seguintes.

* **Saldo de Gols (24,1%)**: O saldo de gols, que mede a diferença entre gols marcados e sofridos, também tem um impacto significativo. Um saldo de gols positivo reflete um time que domina ofensiva e defensivamente, sinalizando maiores chances de sucesso nas próximas partidas. Essa variável indica tanto a capacidade de marcar quanto de defender, ambas essenciais para um bom desempenho contínuo.

* **Gols Sofridos (20,8%)**: A eficiência defensiva de uma equipe, medida pelos gols sofridos, é um fator crítico no modelo. Times com uma defesa sólida têm mais chances de manter ou melhorar suas performances ao longo do campeonato, o que reflete diretamente no número de vitórias futuras.

#### 2.6.3 Análise das Features:

As variáveis com maior importância — **posição na liga**, **pontos por jogo**, **saldo de gols** e **gols sofridos** — capturam o equilíbrio entre eficiência ofensiva e defensiva, além de consistência ao longo da temporada. Esses fatores combinados refletem o desempenho geral da equipe e são excelentes preditores de como ela se sairá nas rodadas restantes. 

Além disso, essas features permitem ao modelo identificar não só os times mais dominantes, mas também aqueles que têm potencial de melhorar sua performance nas rodadas finais. Equipes que ajustam suas estratégias ou que estão em ascensão na tabela podem, com base nessas métricas, se destacar mesmo após um início inconsistente.

A análise de importância das features ajuda a refinar o modelo e pode guiar futuras tomadas de decisão, tanto para prever o desempenho futuro quanto para identificar pontos de melhoria.

In [ ]:
# Obter a importância das features do modelo
feature_importances = model.feature_importances_

# Criar um dataframe para exibir as features e sua importância
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Ordenar as features pela importância
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Exibir as 10 features mais importantes
top_features = feature_importance_df.head(10)

# Plotar as 10 features mais importantes
plt.figure(figsize=(10, 6))
plt.barh(top_features['Feature'], top_features['Importance'], color='skyblue')
plt.xlabel('Importância')
plt.title('Top 10 Features mais importantes no modelo de Random Forest')
plt.gca().invert_yaxis()  # Inverter o eixo para exibir da maior para a menor
plt.show()

# Exibir a tabela com as importâncias das features
top_features

# 3 Comparação com outro modelo

## 3.1 Modelagem para o Problema Com Regressão Linear(Seleção das Principais Features)

Nesta fase, utilizamos a **Regressão Linear** para identificar as variáveis que mais influenciam o número de vitórias futuras de uma equipe em um campeonato de futebol. A **Regressão Linear** é um método estatístico simples, que busca modelar a relação entre uma variável dependente (neste caso, o número de vitórias) e várias variáveis independentes (features selecionadas). Apesar de sua simplicidade, a Regressão Linear pode fornecer insights úteis ao capturar tendências lineares nos dados.

### 3.1 Proposta de Features e Linha de Raciocínio

#### Premissa da Modelagem:

A modelagem parte da premissa de que o desempenho passado de uma equipe ao longo do campeonato pode prever o número de vitórias futuras. As features selecionadas refletem aspectos ofensivos, defensivos e de consistência, que são os principais preditores de sucesso no futebol. A regressão linear é utilizada para captar essas relações de forma direta e linear.

### 3.2 Proposta das Features:

##### Variáveis Utilizadas:
Abaixo estão as principais features selecionadas para o modelo de previsão de vitórias:
- **Desempenho em Rodadas Anteriores**: `matches_played`, `matches_played_home`, `matches_played_away`, `draws`, `losses`, `points_per_game`, `points_per_game_home`, `points_per_game_away`.
- **Posição e Estatísticas de Desempenho Geral**: `league_position`, `goals_scored`, `goals_conceded`, `goal_difference`, `total_goal_count`, `shots_on_target`, `shots_off_target`.
- **Posse de Bola e Controle do Jogo**: `average_possession`.

### 3.3 Explicação do Raciocínio:

1. **Desempenho Passado**: Variáveis como `matches_played`, `points_per_game`, e `league_position` fornecem uma visão clara da consistência de uma equipe. A regressão linear assume que quanto melhor o desempenho prévio de uma equipe, maior a probabilidade de ela continuar vencendo, estabelecendo uma relação linear entre essas variáveis e o número de vitórias.

2. **Eficiência Ofensiva e Defensiva**: Métricas como `goals_scored`, `goals_conceded`, e `goal_difference` são usadas para medir o equilíbrio entre ataque e defesa. A regressão linear assume que times com saldo de gols positivo e uma boa eficiência defensiva tendem a manter um desempenho favorável nas rodadas futuras.

3. **Controle de Jogo**: Variáveis como `average_possession` e o número de chutes refletem como uma equipe controla o jogo. Times que mantêm a posse e criam mais chances de gol são mais propensos a vencer, e a regressão linear tenta capturar essa relação direta.

### 3.4 Normalização das Variáveis:

Antes de aplicar o modelo de regressão linear, utilizamos o método **StandardScaler** para normalizar as variáveis numéricas. Isso garante que todas as variáveis sejam tratadas em uma mesma escala, evitando que variáveis com magnitudes diferentes, como número de chutes e porcentagem de posse de bola, tenham impacto desproporcional no modelo.

### 3.5 Justificativa da Premissa:

- **Posição na Liga e Pontos por Jogo**: Como indicadores de desempenho regular, essas variáveis são preditores naturais para determinar o número de vitórias futuras. Elas captam a capacidade de uma equipe de acumular resultados consistentes ao longo do campeonato.
  
- **Saldo de Gols e Gols Sofridos**: Esses indicadores revelam tanto a força ofensiva quanto defensiva da equipe. A regressão linear tenta estabelecer uma relação proporcional entre um bom saldo de gols e o número de vitórias futuras.
  
- **Histórico de Vitórias**: Equipes que já têm um histórico de vitórias acumuladas tendem a continuar com bom desempenho, e a regressão linear tenta quantificar essa tendência ao prever vitórias adicionais com base no desempenho prévio.

### 3.6 Abordagem Técnica:

A **Regressão Linear** foi escolhida por sua simplicidade e capacidade de fornecer uma linha de base para previsões. Ela busca ajustar um modelo linear que minimize a soma dos erros quadrados entre as previsões e os valores reais. Embora seja uma abordagem mais simples comparada a métodos mais complexos, como o **Random Forest**, ela tem a vantagem de ser rápida e fácil de interpretar. 

No caso de várias variáveis preditoras, a Regressão Linear busca encontrar uma combinação linear das variáveis que melhor explique o número de vitórias. Cada coeficiente da equação linear pode ser interpretado como o impacto que uma variável específica tem no número de vitórias, mantendo as outras constantes.

### 3.7 Previsão e Avaliação:

Após o treinamento do modelo com os dados de treino, o modelo foi testado com o conjunto de teste para fazer previsões. Avaliamos o desempenho do modelo utilizando várias métricas, como:
- **MAE (Mean Absolute Error)**: Mede o erro médio absoluto entre as previsões e os valores reais.
- **R² (Coeficiente de Determinação)**: Avalia quão bem o modelo explica a variabilidade dos dados.
- **MSE (Mean Squared Error)**: Calcula a média dos quadrados dos erros.
- **RMSE (Root Mean Squared Error)**: A raiz quadrada do MSE, oferecendo uma medida de erro mais intuitiva.
- **MedAE (Median Absolute Error)**: A mediana dos erros absolutos, menos sensível a outliers.

In [ ]:
# Definir as colunas significativas para a predição
significant_columns = ['matches_played', 'matches_played_home', 'matches_played_away', 
                       'draws', 'draws_home', 'draws_away', 'losses', 'losses_home', 'losses_away', 'points_per_game', 
                       'points_per_game_home', 'points_per_game_away', 'league_position', 'goals_scored', 'goals_conceded', 
                       'goal_difference', 'total_goal_count', 'shots_on_target', 'shots_off_target', 'average_possession']

# Preparar a matriz de features (X) e a variável alvo (y)
X = dados[significant_columns]
y = dados['wins']

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Treinar o modelo Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = linear_model.predict(X_test)

# Avaliar o modelo com métricas
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
medae = median_absolute_error(y_test, y_pred)

# Prever as vitórias para um time específico
team_id = 16 
team_data = dados[dados['common_name_encoded'] == team_id]

# Verificar se o time tem dados suficientes
if not team_data.empty:
    # Número total de rodadas no campeonato
    total_rounds = 38
    matches_played = team_data['matches_played'].values[0]
    
    # Calcular rodadas restantes
    remaining_rounds = total_rounds - matches_played
    
    # Previsão de vitórias até o momento
    team_X = team_data[significant_columns]
    predicted_wins = linear_model.predict(team_X)
    
    # Ajustar a previsão com base nas rodadas restantes
    predicted_wins_adjusted = (predicted_wins / matches_played) * remaining_rounds
    
    print(f"Previsão de vitórias para o time {team_id} até o final do campeonato:", predicted_wins_adjusted[0])
else:
    print(f"Sem dados para o time {team_id}")

# Exibir as métricas para comparação
(mae, r2, mse, rmse, medae)


### 2.6 Importância das Features no Modelo de Regressão Linear:

#### 2.6.1 Descrição: 
Nesta seção, analisamos a importância de cada variável (feature) no desempenho do modelo de **Regressão Linear**. O modelo linear fornece coeficientes para cada variável, que indicam o impacto direto que essas variáveis têm sobre o número de vitórias previstas. A magnitude e o sinal dos coeficientes permitem identificar quais variáveis contribuem mais (positivamente ou negativamente) para o resultado. Essa análise é essencial para entender como os diferentes fatores influenciam as previsões do modelo e para priorizar ajustes futuros.

#### 2.6.2 Principais Features no Modelo:

* **Posição na Liga (`league_position`)**: Com o maior coeficiente absoluto (-0.294), a posição na liga é a variável mais influente no modelo. Isso significa que times que ocupam posições mais baixas na tabela tendem a acumular menos vitórias. Essa variável resume o desempenho geral de uma equipe ao longo do campeonato, sendo um indicador confiável para prever seu sucesso nas rodadas futuras.

* **Empates em Casa (`draws_home`)**: A segunda variável mais importante, com um coeficiente de -0.214. Empates em casa indicam uma oportunidade perdida de acumular vitórias, o que afeta negativamente o total de vitórias previstas.

* **Total de Empates (`draws`)**: Similar aos empates em casa, o coeficiente de -0.210 para empates totais indica que, quanto mais empates uma equipe acumula, menor é o número de vitórias previstas. O impacto negativo de empates no desempenho é bem claro nesse modelo linear.

* **Jogos Disputados (`matches_played`)**: Com um coeficiente de -0.142, o número de partidas disputadas está inversamente relacionado ao número de vitórias. Isso pode indicar que algumas equipes, mesmo após jogarem muitos jogos, têm dificuldades em converter essas partidas em vitórias.

* **Jogos Disputados em Casa (`matches_played_home`)**: A variável apresenta um coeficiente de -0.135, mostrando que a simples participação em jogos em casa não garante um aumento no número de vitórias.

#### 2.6.3 Análise das Features:

As variáveis com maior impacto no modelo — **posição na liga**, **empates (geral e em casa)** e **número de jogos disputados** — destacam a importância de um desempenho consistente. Times que empatam muito ou ocupam posições mais baixas na tabela tendem a registrar menos vitórias, o que reforça a ideia de que os resultados prévios da equipe são fundamentais para prever seu sucesso futuro.

Além disso, a análise dos coeficientes sugere que apenas disputar jogos (especialmente em casa) não é suficiente para garantir vitórias, e que fatores como a capacidade de evitar empates e subir na tabela são mais relevantes. Isso indica que o modelo linear captura uma relação clara entre o desempenho passado e as vitórias futuras, com uma ênfase nos resultados mais diretos, como a posição na liga.

Equipes que mostram um bom desempenho global, com poucas derrotas e uma boa posição na liga, têm maior probabilidade de continuar acumulando vitórias. Essa análise também destaca a necessidade de as equipes evitarem empates para maximizar suas chances de sucesso.

In [ ]:
# Extrair os coeficientes da Regressão Linear
coefficients = linear_model.coef_

# Criar um DataFrame para visualizar os coeficientes
coefficients_df = pd.DataFrame({
    'Feature': significant_columns,
    'Coefficient': coefficients
})

# Ordenar pelos coeficientes (valores absolutos)
coefficients_df = coefficients_df.reindex(coefficients_df.Coefficient.abs().sort_values(ascending=False).index)

top_coefficients = coefficients_df.head(10)

plt.figure(figsize=(10, 6))
plt.barh(top_coefficients['Feature'], top_coefficients['Coefficient'], color='skyblue')
plt.xlabel('Coeficiente')
plt.title('Top 10 Coeficientes das Variáveis no Modelo de Regressão Linear')
plt.gca().invert_yaxis()  # Inverter o eixo para exibir da maior para a menor
plt.show()

print(coefficients_df)


## 3. Comparação dos Modelos

### 3.1 Previsão de Vitórias para o Time 16:
- **Random Forest Regressor**: 16.11 vitórias
- **Regressão Linear**: 16.11 vitórias

Ambos os modelos previram **exatamente** o mesmo número de vitórias (16.11) para o time 16 até o final do campeonato, o que demonstra que, apesar de suas abordagens diferentes, os dois modelos chegaram a resultados semelhantes para essa tarefa de previsão específica.

### 3.2 Métricas de Avaliação:

| Métrica            | Random Forest        | Regressão Linear     |
|--------------------|----------------------|----------------------|
| **MAE**            | 0.365                | 0.402                |
| **R² Score**       | 0.881                | 0.854                |
| **MSE**            | 0.206                | 0.252                |
| **RMSE**           | 0.454                | 0.502                |
| **MedAE**          | 0.425                | 0.299                |

### 3.3 Análise Comparativa:

#### 3.3.1. **Previsão para o time 16**:
- Apesar de serem baseados em algoritmos diferentes, ambos os modelos previram o mesmo número de vitórias (16.11) para o time 16 até o final do campeonato. Isso pode sugerir que as variáveis selecionadas e os dados de entrada são suficientemente informativos para que ambos os modelos cheguem a uma conclusão semelhante. No entanto, a maneira como cada modelo chegou a essa previsão varia, o que reflete nas diferenças nas métricas de avaliação.

#### 3.3.2. **Erro Médio Absoluto (MAE)**:
- **Random Forest Regressor** apresentou um **MAE menor** (0.365 contra 0.402 para Regressão Linear). O **MAE (Mean Absolute Error)** mede a média dos erros absolutos entre os valores previstos e os valores reais. Um **MAE menor** sugere que, em média, o Random Forest fez previsões mais precisas em comparação com a Regressão Linear. Isso indica que, embora ambos os modelos tenham feito previsões semelhantes para o time 16, o Random Forest foi ligeiramente mais preciso em prever os resultados para todos os times no conjunto de teste.

#### 3.3.3. **R² Score**:
- O **Random Forest Regressor** apresentou um **R² Score** de 0.881, enquanto a **Regressão Linear** teve um **R² Score** de 0.854. O **R² Score** mede a proporção da variância nos dados que o modelo consegue explicar. Um valor mais próximo de 1 indica que o modelo é melhor em capturar a variabilidade dos dados. Embora ambos os modelos tenham um desempenho bom, o Random Forest explica uma fração maior da variação total nos dados. Isso significa que, no geral, o Random Forest é um modelo um pouco mais eficaz em prever as vitórias com base nas features fornecidas.

#### 3.3.4. **Erro Quadrático Médio (MSE) e Raiz do Erro Quadrático Médio (RMSE)**:
- O **Random Forest Regressor** também teve um **MSE (Mean Squared Error)** menor (0.206 contra 0.252 para Regressão Linear) e, consequentemente, um **RMSE (Root Mean Squared Error)** mais baixo (0.454 contra 0.502). Esses erros quadráticos penalizam desvios maiores mais fortemente do que erros menores, e por isso um **MSE** menor sugere que o Random Forest teve erros mais distribuídos e com menos desvios significativos em relação à Regressão Linear. O **RMSE**, sendo a raiz quadrada do MSE, traduz essa métrica para o mesmo espaço de unidades das previsões, tornando o valor mais interpretável. Novamente, o menor **RMSE** do Random Forest indica que ele teve um desempenho mais robusto, especialmente no tratamento de grandes desvios.

#### 3.3.5. **Erro Mediano Absoluto (MedAE)**:
- A **Regressão Linear** obteve um **MedAE (Median Absolute Error)** de 0.299, significativamente menor do que o valor de 0.425 do **Random Forest Regressor**. O **MedAE** mede a mediana dos erros absolutos, sendo menos sensível a outliers em comparação com o **MAE**. Isso sugere que, na mediana, a Regressão Linear teve erros mais consistentes e menos dispersos, enquanto o Random Forest pode ter sido influenciado por alguns outliers maiores.

### 3.4 Análise dos Coeficientes e Importâncias das Features:

#### 3.4.1 Regressão Linear:
- A **Regressão Linear** nos fornece coeficientes claros que indicam a magnitude e a direção (positiva ou negativa) do impacto de cada feature nas previsões.
- Por exemplo, a **posição na liga** teve o maior coeficiente negativo (-0.294), o que significa que quanto pior a posição na liga (quanto mais alto o número, indicando uma posição mais baixa), menor o número de vitórias previstas.
- Os **empates em casa** também tiveram um impacto negativo significativo (-0.214), sugerindo que empates em casa são menos valiosos para a previsão de vitórias do que outros fatores.

#### 3.4.2 Random Forest Regressor:
- O **Random Forest Regressor** não fornece coeficientes, mas podemos interpretar a **importância das features**. A **posição na liga** foi a feature mais importante (25.5%), o que corrobora com a Regressão Linear, que também mostrou um impacto significativo dessa variável.
- **Pontos por jogo** (24.1%) e **saldo de gols** (24.1%) foram outras variáveis de destaque no Random Forest, refletindo sua importância na previsão do número de vitórias.

### 3.5 Conclusão:

- **Desempenho Geral**: Embora ambos os modelos tenham fornecido previsões semelhantes para o time 16, o **Random Forest Regressor** teve um desempenho **geralmente melhor** em termos de precisão, com **menor MAE**, **MSE** e **RMSE**. Isso indica que ele é mais eficaz em capturar variações mais complexas nos dados.
- **Consistência**: A **Regressão Linear** mostrou ser um modelo **mais consistente**, como evidenciado pelo seu **MedAE** mais baixo. Isso significa que, em termos medianos, as previsões da Regressão Linear foram mais uniformes, com menos grandes desvios em comparação ao Random Forest.
- **Escolha do Modelo**: Se o objetivo for um modelo que tenha **maior precisão global** e capture melhor as variações nos dados, o **Random Forest Regressor** seria a melhor escolha. No entanto, se a prioridade for um modelo **simples, interpretable** e com previsões **mais consistentes**, a **Regressão Linear** pode ser preferida.

Em resumo, o **Random Forest Regressor** oferece um desempenho levemente superior para este problema, mas a **Regressão Linear** ainda é uma opção viável, especialmente devido à sua simplicidade e interpretabilidade.

## 4. Explicabilidade do Modelo com SHAP

Nesta seção, utilizaremos o SHAP (SHapley Additive exPlanations) para explicar o modelo Random Forest Regressor que prediz o número de vitórias de times da Série A do Campeonato Brasileiro. O SHAP é uma ferramenta poderosa para a explicação de modelos de Machine Learning, baseada na teoria dos valores de Shapley, que mede o impacto de cada feature nas previsões de forma global e individual.

### 4.1 Configuração e Cálculo dos Valores SHAP

Primeiro, configuramos o SHAP e calculamos os valores de Shapley para o conjunto de teste:

In [67]:
# Cria o objeto Explainer para o modelo
explainer = shap.Explainer(model, X_train)

# Calcula os valores SHAP para o conjunto de teste
shap_values = explainer(X_test)

Neste trecho, criamos um objeto `Explainer` usando o modelo treinado e o conjunto de dados de treino (`X_train`). Em seguida, calculamos os valores de SHAP para o conjunto de teste (`X_test`), que nos permitirão avaliar o impacto de cada feature nas previsões do modelo.

### 4.2 Análise Global da Importância das Features

Para entender quais features têm o maior impacto no modelo como um todo, utilizamos o gráfico de resumo:


In [ ]:
# Gráfico de resumo para visualizar a importância das features
shap.summary_plot(shap_values, X_test, plot_type="bar")

#### Interpretação:
- **`league_position`**: É a variável com maior impacto nas previsões de vitórias, sugerindo que a posição atual de um time na liga é fundamental para prever o número de vitórias.
- **`goal_difference`** e **`points_per_game`**: Essas variáveis também são altamente influentes, indicando que tanto a eficiência de gols quanto a consistência de pontos acumulados por jogo desempenham papéis cruciais nas previsões.
- **Outras Features**: Variáveis como `goals_conceded` e `draws_home` têm menos impacto, mas ainda contribuem para as previsões.

O gráfico de barras ordena as variáveis por importância, ajudando a identificar quais variáveis devem receber mais atenção ao ajustar ou interpretar o modelo.

### 4.3 Análise de Dependência de Feature Específica

O gráfico de dependência nos permite ver como a variação de uma feature específica afeta as previsões. Vamos analisar o impacto da feature `points_per_game`:

In [ ]:
shap.dependence_plot("points_per_game", shap_values.values, X_test)


#### Interpretação:
- No gráfico, o eixo X representa os valores de `points_per_game`, enquanto o eixo Y mostra os valores de SHAP, que indicam o impacto desse atributo na previsão.
- Cada ponto representa uma observação (um time), e a cor dos pontos mostra a relação com outra variável, neste caso, `matches_played`.
- Observa-se que à medida que `points_per_game` aumenta, o impacto na previsão de vitórias também tende a ser positivo. Times com mais pontos por jogo são mais propensos a ter um número maior de vitórias.

### 4.4 Explicação de Previsões Individuais

Para entender como o modelo toma decisões para observações específicas, utilizamos o gráfico de força para uma previsão individual. Este gráfico detalha o efeito de cada feature na previsão final:


In [ ]:
shap.force_plot(explainer.expected_value, shap_values.values[1], X_test.iloc[1,:], matplotlib=True)

Neste gráfico de força:
- **Setas para a Direita (vermelhas)**: Indicadores de features que aumentam a previsão de vitórias.
- **Setas para a Esquerda (azuis)**: Indicadores de features que reduzem a previsão de vitórias.
- **Valor Base**: Representa a média de previsões do modelo. A contribuição das features ajusta esse valor para obter a previsão final.

### 4.5 Conclusão e Insights sobre o Uso do SHAP

A análise com SHAP fornece uma explicabilidade detalhada do modelo Random Forest Regressor, destacando:
- **Importância das Features**: As variáveis `league_position`, `goal_difference` e `points_per_game` são determinantes nas previsões, indicando que a posição na liga e a consistência de pontos são as métricas que o modelo mais utiliza.
- **Interações de Variáveis**: O gráfico de dependência revela como variáveis como `points_per_game` interagem com outras, como `matches_played`, ajudando a identificar relações complexas entre features.
- **Explicabilidade Individual**: Com o gráfico de força, entendemos como o modelo faz previsões para cada time, detalhando o peso de cada variável, o que ajuda a explicar por que certos times têm previsões de vitórias mais altas ou mais baixas.

## 5 Apresentação do Modelo Candidato

Dado que o objetivo é prever um valor numérico (como a quantidade de vitórias de um time em uma temporada), o modelo escolhido foi a **Regressão Linear**. A **Regressão Linear** é amplamente utilizada em problemas de regressão simples e oferece uma abordagem interpretável para prever variáveis contínuas. Embora seja uma abordagem simples, ela permite entender claramente como cada feature impacta diretamente a variável alvo (número de vitórias).

* A **Regressão Linear** busca ajustar uma linha (ou hiperplano, no caso de múltiplas variáveis) que minimiza a soma dos erros quadrados entre as previsões e os valores reais. Este modelo é fácil de interpretar, pois fornece coeficientes que indicam o impacto de cada variável na predição.

* Além disso, o modelo de regressão linear permite uma visualização direta do relacionamento linear entre as variáveis preditoras e a variável alvo, facilitando a análise de quais fatores são mais relevantes para a previsão de vitórias ou gols.

### 5.1 Variáveis Significativas

As variáveis selecionadas para o modelo incluem fatores relacionados ao desempenho histórico do time, tais como:

- **matches_played**: Partidas jogadas até o momento
- **draws, losses**: Número de empates e derrotas
- **goals_scored, goals_conceded**: Gols marcados e sofridos pelo time
- **shots_on_target, shots_off_target**: Finalizações no alvo e fora do alvo
- **average_possession**: Percentual médio de posse de bola durante as partidas
- **league_position**: Posição atual do time na liga

Essas variáveis foram escolhidas com base em sua importância esperada no desempenho de um time ao longo da temporada. O modelo de Regressão Linear busca capturar como esses fatores se correlacionam com o número de vitórias.

### 5.2 Divisão dos Dados

Os dados foram divididos em conjuntos de treino e teste, com **80% dos dados** sendo utilizados para treinar o modelo e **20%** para testar sua precisão. A divisão foi feita de forma aleatória para garantir que o conjunto de teste seja representativo da variabilidade dos dados.

```python
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

### 5.3 Treinamento do Modelo

O modelo de **Regressão Linear** foi treinado utilizando o conjunto de treino. Esse processo envolve a busca da melhor linha de ajuste (ou hiperplano) que minimize o erro entre as previsões e os valores reais.

```python
model = LinearRegression()
model.fit(X_train, y_train)
```

A simplicidade da Regressão Linear permite que o modelo seja treinado rapidamente, o que é uma vantagem em termos de custo computacional, especialmente em problemas que não exigem alta complexidade.

### 5.4 Métricas de Avaliação

Para avaliar o desempenho do modelo, foram utilizadas as seguintes métricas de regressão:

- **Mean Absolute Error (MAE)**: Mede o erro médio absoluto entre as previsões e os valores reais, sendo uma métrica intuitiva que reflete a magnitude dos erros.
- **Mean Squared Error (MSE)**: Penaliza desvios maiores, sendo útil para capturar grandes erros nas previsões.
- **Root Mean Squared Error (RMSE)**: É a raiz quadrada do MSE, trazendo a métrica para a mesma escala dos dados originais, facilitando sua interpretação.
- **R² (Coeficiente de Determinação)**: Indica a proporção da variabilidade dos dados que é explicada pelo modelo. Quanto mais próximo de 1, melhor o ajuste do modelo aos dados.

Essas métricas fornecem uma visão abrangente do quão bem o modelo de Regressão Linear foi capaz de prever as vitórias com base nas variáveis selecionadas.

### 5.5 Conclusão

A **Regressão Linear** provou ser um modelo adequado para a previsão de vitórias, oferecendo uma abordagem **simples, direta e interpretável**. Apesar de ser um modelo básico, ele mostrou bom desempenho nas métricas de avaliação, capturando de forma eficaz o relacionamento linear entre as variáveis de desempenho dos times e o número de vitórias.

# 6 Referências

Está é uma seção de referências com relação as bibliotecas que utilizamos ao longo deste arquivo

NUMPY. NumPy Documentation. Disponível em: <https://numpy.org/doc/>.

‌PANDAS. pandas documentation. Disponível em: <https://pandas.pydata.org/docs/>.

MATPLOTLIB. Matplotlib: Python plotting — Matplotlib 3.3.4 documentation. Disponível em: <https://matplotlib.org/stable/index.html>.

‌SEABORN. seaborn: statistical data visualization — seaborn 0.9.0 documentation. Disponível em: <https://seaborn.pydata.org/>.

‌SciPy documentation — SciPy v1.8.1 Manual. Disponível em: <https://docs.scipy.org/doc/scipy/>.

‌PYTHON SOFTWARE FOUNDATION. Math — Mathematical Functions — Python 3.8.3rc1 Documentation. Disponível em: <https://docs.python.org/3/library/math.html>.

‌SCIKIT-LEARN. scikit-learn: machine learning in Python. Disponível em: <https://scikit-learn.org/stable/>.